In [1]:
import numpy as np
import pandas as pd

In [2]:
def read_file(filename):
    with open(filename, 'r') as f:
        content = f.read()
        
    return content

In [35]:
def load_data():
    data = {}
    for index in range(3):
        X_train = pd.read_csv('Xtr%s_mat50.csv' % index, delimiter=' ', names=range(0,50)).values
        X_test = pd.read_csv('Xte%s_mat50.csv' % index, delimiter=' ', names=range(0,50)).values
        labels = pd.read_csv('Ytr%s.csv' % index, names=('Id', 'Bound'), skiprows=[0], delimiter=',')

        data[index] = {
            'X_train': X_train,
            'y_train': labels['Bound'].values,
            'X_test': X_test,
            'ids': labels['Id'].values,
        }
        
    return data

DATA = load_data()

## Kernel ridge regression 

In [36]:
from ridge_regression import get_ridge_prediction, kernel_ridge_regression

In [37]:
def linear_K(x1, x2):
    return np.matmul(x1.T, x2)

In [38]:
alpha = kernel_ridge_regression(linear_K, DATA[0]['X_train'], DATA[0]['y_train'], 0.1)
alpha.shape

(2000,)

In [39]:
pred = get_ridge_prediction(linear_K, DATA[0]['X_train'], DATA[0]['X_train'], alpha)
pred.shape

(2000,)

In [40]:
def get_precision(y_pred, y_true):
    return np.sum(y_pred == y_true) / len(y_pred)

get_precision(pred, DATA[0]['y_train'])

0.63949999999999996

In [42]:
def produce_results(preds, ids):
    data = [
        '%s,%s' % (id, pred) for pred, id in zip(preds, ids)
    ]
    
    with open('submission.csv', 'w') as f:
        f.write('\n'.join(data))

In [43]:
produce_results(get_ridge_prediction(linear_K, DATA[0]['X_train'], DATA[0]['X_test'], alpha), DATA[0]['ids'])

In [54]:
def train_and_pred():
    preds = []
    ids = []
    for k in range(3):
        alpha = kernel_ridge_regression(linear_K, DATA[k]['X_train'], DATA[k]['y_train'], 0.1)
        pred = get_ridge_prediction(linear_K, DATA[k]['X_train'], DATA[k]['X_test'], alpha)
        
        preds += list(pred)

    produce_results(preds, range(3000))

In [55]:
train_and_pred()